In [1]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch==2.0.1
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [23]:
notebook_login()

# LOAD FALCON MODEL & TOKENIZER

In [4]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [6]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainables%: 0.13058363808693696


# Test original model

In [8]:
prompt = """
<human>:
<assistant>:
""".strip()

In [9]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [10]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<human>:
<assistant>:
<
CPU times: user 1min 13s, sys: 346 ms, total: 1min 13s
Wall time: 1min 17s


# Prep dataset

In [11]:
data = load_dataset("csv", data_files="/content/constitution_train.csv")

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'output'],
        num_rows: 933
    })
})

In [13]:
data["train"][0]

{'prompt': "What is the definition of 'Panchayat' in the Constitution of India?",
 'output': "In the Constitution of India, a 'Panchayat' is defined as an institution (by whatever name called) of self-government constituted under article 243B, for the rural areas."}

In [14]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["prompt"]}
<assistant>: {data_point["output"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [15]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

Map:   0%|          | 0/933 [00:00<?, ? examples/s]

In [16]:
data

Dataset({
    features: ['prompt', 'output', 'input_ids', 'attention_mask'],
    num_rows: 933
})

# Finetune the model

In [17]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      # fp16=False,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.340200
2,2.170400
3,1.970600
4,2.391000
5,1.929100
6,1.941900
7,2.307700
8,2.093000
9,2.212600
10,2.002900


TrainOutput(global_step=233, training_loss=1.464406299232925, metrics={'train_runtime': 3037.5627, 'train_samples_per_second': 0.307, 'train_steps_per_second': 0.077, 'total_flos': 1952931669072384.0, 'train_loss': 1.464406299232925, 'epoch': 1.0})

# Save trained model

In [18]:
model.save_pretrained("trained-model")

In [26]:
PEFT_MODEL = "HarshSinyal/Constitution-India-Falcon-Sharded-7B"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

CommitInfo(commit_url='https://huggingface.co/HarshSinyal/Constitution-India-Falcon-Sharded-7B/commit/0ae3c40c0c4e0bccb535ce2fa6014f603a128d16', commit_message='Upload model', commit_description='', oid='0ae3c40c0c4e0bccb535ce2fa6014f603a128d16', pr_url=None, pr_revision=None, pr_num=None)

In [27]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

Some weights of FalconForCausalLM were not initialized from the model checkpoint at vilsonrodrigues/falcon-7b-instruct-sharded and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Run the finetuned model

In [28]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [38]:
%%time
device = "cuda:0"

prompt = """
<human>: What are the civil rights of a man according to Article 12?
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.no_grad():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

tokenizer.decode(outputs[0], skip_special_tokens=True)

'<human>: What are the civil rights of a man according to Article 12?\n<assistant>: According to Article 12, every man has the right to freedom of speech and expression, the right to practice any religion, and the right to move freely throughout the territory of India. Additionally, every man has the right to be free from discrimination on the basis of religion, race, caste, sex, or place of birth. These rights are subject to the provisions of any law made by Parliament. The State shall take steps to ensure the equality of men and women in the matter of marriage, divorce, adoption, and inheritance. The State shall also take steps to ensure that the rights of women are not restricted by any custom or practice. The State shall also take steps to ensure that the rights of women are not restricted by any custom or practice. The State shall also take steps to ensure that the rights of women are not restricted by any custom or practice. The State shall also take steps to ensure that the righ